In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from soynlp.word import WordExtractor

In [4]:
data = pd.read_csv('/aiffel/aiffel/AIFFELTHON/data/train_RI100_2000.csv')

In [2]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면']


In [7]:
sentences = data.non_label_sentence
sentences = [sen for sen in sentences]

from soynlp.tokenizer import LTokenizer

word_extractor = WordExtractor(
    min_frequency=100, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

training was done. used memory 0.544 Gbry 0.409 Gb
all cohesion probabilities was computed. # words = 1877
all branching entropies was computed # words = 52472
all accessor variety was computed # words = 52472


In [10]:
data['tokenized'] = data['non_label_sentence'].apply(tokenizer.tokenize)
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
data

,label_sentence,non_label_sentence,class,binary_class,tokenized
0,1:안녕하세요 친절 상담원 입니다\n2:저 수강중인게 있는데 일시정지를 하고 싶어서...,친절\n저 수강중인게 있는데 일시정지를 하고 싶어서요\n일시정지는 강좌 당 일회 가...,일반 대화,일반 대화,"[친절, 저, 수강중인게, 있는데, 일시정지를, 하고, 싶어, 서요, 일시정지는, ..."
1,1:욕실 발 매트 새로 사고 싶다\n2:왜 망가졌나\n1:지금 쓰는 규조토 물을 이...,욕실 발 매트 새로 사고 싶다\n왜 망가졌나\n지금 쓰는 규조토 물을 이제 더 이상...,일반 대화,일반 대화,"[욕실, 발, 매트, 새로, 사고, 싶다, 왜, 망가졌나, 지금, 쓰는, 규조토, ..."
2,1:안녕하세요 입니다 민원인께서 문의 주셨습니다\n1:아산 음봉에 율지천 하천 점용...,민원인께서 문의 주셨습니다\n아산 음봉에 율지천 하천 점용 관련으로 하천안전과 연결...,일반 대화,일반 대화,"[민원인께서, 문의, 주셨습니다, 아산, 음봉에, 율지천, 하천, 점용, 관련, 으..."
3,1:반려동물을 씻기는거 말고도 고난이도가 뭐가 있어\n2:발톱 깎이는거야\n1:그런...,반려동물을 씻기는거 말고도 고난이도가 뭐가 있어\n발톱 깎이는거야\n그런데 왜 힘들...,일반 대화,일반 대화,"[반려동물을, 씻기는거, 말고, 고난이도가, 뭐가, 있어, 발톱, 깎이는거야, 그런..."
4,1:데이트 몰아서 하는거 아닌가여\n2:일주일에 두 번만 보면 돼\n2:더 보면 싸...,데이트 몰아서 하는거 아닌가여\n일주일에 두 번만 보면 돼\n더 보면 싸운다\n노...,일반 대화,일반 대화,"[데이, 트, 몰아서, 하는, 거, 아닌가, 여, 일주일, 두, 번만, 보면, 돼,..."
...,...,...,...,...,...
9735,1:준하야 넌 대가리가 왜 이렇게 크냐\n2:내 머리가 뭐\n1:밥 먹으면 대가리만...,준 우승자 하야 스즈키 넌 플리트 대가리 가 왜 과연 이렇게 크냐\n내 내고 머리 ...,기타 괴롭힘 대화,공격 대화,"[준, 우승자, 하야, 스즈키, 넌, 플리트, 대가리, 왜, 과연, 이렇게, 크냐,..."
9736,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내 내고 가 지금 현재 너 에드 아들 딸 김길준 데리 거느리 고 있어 살리고 싶으면...,갈취 대화,공격 대화,"[내, 내고, 지금, 현재, 너, 에드, 아들, 딸, 김길준, 데리, 거느리, 있어..."
9737,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나 나의 는 씨 정씨 같은 사람 젊은이 보면 참 신기하더라 어떻게 저렇게 살 살아오...,직장 내 괴롭힘 대화,공격 대화,"[나, 나의, 씨, 정씨, 같은, 사람, 젊은, 보면, 참, 신기, 하더라, 어떻게..."
9738,1:누구 맘대로 여기서 장사하래\n2:이게 무슨 일입니까\n1:남의 구역에서 장사하...,누구 어디 맘대로 여기 알리 서 장사 태수 하래\n이 그러 게 무슨 어찌 일 입 입...,갈취 대화,공격 대화,"[누구, 어디, 맘대로, 여기, 알리, 서, 장사, 태수, 하래, 그러, 무슨, 어..."


In [11]:
conversation = data['tokenized'].values

tokenizer_tf = Tokenizer()
tokenizer_tf.fit_on_texts(conversation)
len(tokenizer_tf.word_index)

63184

In [14]:
threshold = 2
total_cnt = len(tokenizer_tf.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer_tf.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('전체 단어 집합에서 희귀 단어를 뺀 크기 : ', total_cnt - rare_cnt)
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 63184
등장 빈도가 1번 이하인 희귀 단어의 수: 34243
전체 단어 집합에서 희귀 단어를 뺀 크기 :  28941
단어 집합에서 희귀 단어의 비율: 54.19568245125348
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 4.032103280624639


In [32]:
vocab_size = 30000#total_cnt - rare_cnt
tokenizer_tf = Tokenizer(vocab_size) 
tokenizer_tf.fit_on_texts(conversation)
X_data = tokenizer_tf.texts_to_sequences(conversation)

In [33]:
padded = pad_sequences(X_data)
print(padded)

[[    0     0     0 ...  1758  1086   328]
 [    0     0     0 ...  2039     4  7205]
 [    0     0     0 ...  1042  1414 28973]
 ...
 [    0     0     0 ...     7     2   263]
 [    0     0     0 ...  4042  6015   803]
 [    0     0     0 ...  7186  1719    15]]


In [46]:
train_data = padded
train_label = data['class']

labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

train_label = train_label.apply(lambda x : labels[x])

In [48]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22, stratify=train_label)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22, stratify=test_Y)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

7792 974 974
7792 974 974


In [49]:
train_Y

4057    2
3794    0
4004    0
4715    3
5678    3
       ..
8801    3
7180    1
8874    1
9099    3
7062    3
Name: class, Length: 7792, dtype: int64

In [37]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [38]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [59]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = train_X.shape[1]

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(len(labels), activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [52]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [60]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    train_X, train_Y, batch_size=32, epochs=10, validation_data=(valid_X, valid_Y), callbacks=[es]
)

Epoch 1/10
244/244 [==============================] - 4s 11ms/step - loss: 1.3683 - accuracy: 0.4026 - val_loss: 0.6157 - val_accuracy: 0.8439
Epoch 2/10
244/244 [==============================] - 2s 10ms/step - loss: 0.3670 - accuracy: 0.8818 - val_loss: 0.2669 - val_accuracy: 0.9066
Epoch 3/10
244/244 [==============================] - 2s 10ms/step - loss: 0.1268 - accuracy: 0.9660 - val_loss: 0.1882 - val_accuracy: 0.9374
Epoch 4/10
244/244 [==============================] - 2s 10ms/step - loss: 0.0452 - accuracy: 0.9883 - val_loss: 0.2309 - val_accuracy: 0.9405
Epoch 5/10
244/244 [==============================] - 2s 10ms/step - loss: 0.0199 - accuracy: 0.9958 - val_loss: 0.1936 - val_accuracy: 0.9528
Epoch 00005: early stopping


In [62]:
test_file_path = '/aiffel/aiffel/AIFFELTHON/data/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [63]:
import re


def remove_punctuation(x):
  x = x.strip()
  x = re.sub("[^가-힣0-9]+", " ", x)
  x = re.sub("[ ]+", " ", x)
  x = x.strip()
  return x

test_data["text"] = test_data['text'].apply(lambda x : remove_punctuation(x))
test_data.head()

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 2...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...


In [75]:
sentences = [sen for sen in test_data['text']]
sentences = [tokenizer.tokenize(sen) for sen in sentences]
sequences = tokenizer_tf.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post', maxlen=maxlen)

In [73]:
padded

array([[ 578,  692,   54, ...,    0,    0,    0],
       [  37, 6860,  128, ...,    0,    0,    0],
       [   3,   57,  100, ...,    0,    0,    0],
       ...,
       [   1,    3, 1433, ...,    0,    0,    0],
       [   1,    3,   78, ...,    0,    0,    0],
       [  85,   22,   47, ...,    0,    0,    0]], dtype=int32)

In [76]:
pred = model.predict(padded)

from sklearn.metrics import classification_report

pred_label = []

for i in range(len(pred)):
    pred_label.append((str)(np.argmax(pred[i])).zfill(2))

In [78]:
pred_label

['03',
 '02',
 '02',
 '04',
 '04',
 '00',
 '02',
 '03',
 '03',
 '01',
 '00',
 '04',
 '02',
 '02',
 '02',
 '03',
 '04',
 '03',
 '00',
 '03',
 '00',
 '03',
 '03',
 '03',
 '01',
 '02',
 '03',
 '04',
 '02',
 '03',
 '01',
 '00',
 '03',
 '03',
 '03',
 '03',
 '03',
 '00',
 '03',
 '01',
 '00',
 '03',
 '02',
 '01',
 '03',
 '02',
 '00',
 '03',
 '04',
 '00',
 '01',
 '03',
 '03',
 '02',
 '02',
 '03',
 '03',
 '04',
 '03',
 '03',
 '03',
 '03',
 '02',
 '03',
 '02',
 '00',
 '03',
 '03',
 '03',
 '03',
 '03',
 '02',
 '03',
 '02',
 '03',
 '04',
 '03',
 '04',
 '02',
 '03',
 '03',
 '00',
 '02',
 '00',
 '03',
 '02',
 '03',
 '04',
 '03',
 '02',
 '03',
 '02',
 '00',
 '03',
 '03',
 '02',
 '03',
 '02',
 '01',
 '02',
 '03',
 '01',
 '03',
 '00',
 '03',
 '02',
 '03',
 '03',
 '03',
 '00',
 '04',
 '03',
 '03',
 '02',
 '03',
 '01',
 '03',
 '02',
 '02',
 '03',
 '02',
 '03',
 '03',
 '01',
 '01',
 '01',
 '02',
 '03',
 '01',
 '03',
 '03',
 '03',
 '03',
 '03',
 '01',
 '00',
 '04',
 '00',
 '03',
 '03',
 '03',
 '03',
 '01',

In [79]:

submission = test_data.assign(CLASS=pred_label)
submission = submission.rename(columns={'CLASS':'class'})

In [80]:
submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()
submission

,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
class,03,02,02,04,04,00,02,03,03,01,...,03,04,00,01,03,02,03,03,02,03


In [82]:
import json

submission_file_path = '/aiffel/aiffel/AIFFELTHON/submission/submission_transformer_RI100_2000.json'
result = submission.to_json(submission_file_path)

with open(submission_file_path) as f:
    parsed = json.load(f)

with open(submission_file_path, 'w') as f:
    json.dump(parsed, f, indent=4)